In [1]:
# collect data from Twitter using Twitter API

In [3]:
# !pip install tweepy
import tweepy as tw
print(tw.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 KB 6.1 MB/s eta 0:00:00
4.10.0


In [15]:
# !pip install pandas
import pandas as pd
import json
import time
import csv

In [28]:
# assign the values accordingly

In [29]:
client = tw.Client(
    wait_on_rate_limit = True,
    consumer_key = consumer_key,
    consumer_secret = consumer_secret,
    access_token = access_token,
    access_token_secret = access_token_secret,
    bearer_token = bearer_token,
)

In [30]:
# client = tw.Client(bearer_token)
client

In [32]:
username_list = ["ABC","ABCPolitics","ABCNewsLive"]

# ,"BBCWorld","BBCBreaking","BBCNews","BBCNorthAmerica",
#                  "CBSNews","CBSPolitics","CBSNewsPoll","CNN","cnnbrk","CNNPolitics","FoxNews","foxnewspolitics",
#                  "euronews","MSNBC","NBCNews","NBCPolitics","nytimes","nytpolitics","NPR","nprpolitics","SkyNews",
#                  "USATODAY","usatodayDC","WSJ","WSJusnews","washingtonpost"

In [46]:
# users = client.get_users(usernames=username_list, user_fields=["created_at","location","protected","public_metrics","verified"])
# users

def get_users_details(usernames):
        users_details = client.get_users(usernames=usernames, user_fields=["created_at","location","protected","public_metrics","verified"])
        return users_details

users = get_users_details(username_list)
users

In [ ]:
for user in users.data:
    print(user["id"])
    print(user["created_at"])

In [ ]:
users.data[0]["location"]

In [1]:
# Save data as dictionary
users_dict = users.json() 

# Extract "data" value from dictionary
users_data = users_dict['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(users_data) 

df

NameError: name 'users' is not defined

In [38]:
# Create empty dataframe
data_users = pd.DataFrame()

# Iterate through users_group and append data to the data_users dataframe
for i in users.data:
  temp_data = pd.json_normalize( i.data ,  sep = "_")
  data_users = data_users.append( temp_data , ignore_index=True )

<ipython-input-38-1c7f32d9186a>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_users = data_users.append( temp_data , ignore_index=True )
<ipython-input-38-1c7f32d9186a>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_users = data_users.append( temp_data , ignore_index=True )
<ipython-input-38-1c7f32d9186a>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_users = data_users.append( temp_data , ignore_index=True )


In [14]:
df = pd.DataFrame(response['users'])

NameError: name 'response' is not defined

In [51]:
# Get User's Tweets

# This endpoint/method returns Tweets composed by a single user, specified by
# the requested user ID

user_id = 28785486

response = client.get_users_tweets(user_id)

# By default, only the ID and text fields of each Tweet will be returned
for tweet in response.data:
    print(tweet.id)
    print(tweet.text)

# By default, the 10 most recent Tweets will be returned
# You can retrieve up to 100 Tweets by specifying max_results
response = client.get_users_tweets(user_id, max_results=100)

1551640435164061702
Firefighters have managed to contain 10% of the blaze, officials say. https://t.co/yRdYAhx6co
1551640001863159808
BREAKING: Paul Sorvino, best known for starring in "Goodfellas" and "Law &amp; Order," has passed away. He was 83. https://t.co/YRnNhktlij https://t.co/7duORP636e
1551639937149272064
17 states are now offering tax-free holidays for school supplies, cutting sales tax ahead of the start of school.

Learn more: https://t.co/PIx6PAJQd9
1551639716621164547
As the heat hovering over much of the country continues, residents on the East Coast will soon have another extreme weather event to contend with: severe storms. https://t.co/DMHY31puar
1551634902977839109
Family Dollar shoppers should check their medicine cabinets and bathrooms for any recently recalled products. https://t.co/P8EGjd1uTx
1551632233345286145
NEW: The Georgia DA leading Fulton County's 2020 election probe has been barred from investigating one of the state's 16 alleged "fake electors" due to 

In [13]:
# search_words = "Joe Biden" # This will be changed to Donald Trump when we retrieve tweets related to him.
# date_since = "2020-10-27"
# tweets = tw.Cursor(api.search_tweets, q=search_words, lang="en").items(10)
# tweets

In [14]:
# tweet_details = [[tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]
# tweet_df = pd.DataFrame(data=tweet_details, columns=["geo","text","user","location"])
# tweet_df.head()

In [ ]:
# df.to_csv('data.csv')